# Clinical Care Coordination System - Learning Guide

## Google ADK + BioGPT Multi-Agent Framework

This notebook explains **every file** in your project, **why it's important**, and the **order you should study them**.

---

## 🎯 Project Overview

Your capstone project demonstrates:
1. **Custom ML Model** - Your fine-tuned BioGPT for medical text
2. **Multi-Agent System** - 6 specialized agents using Google ADK
3. **Agent Patterns** - Sequential, Parallel, and Loop patterns
4. **Tool Integration** - Custom tools for clinical operations
5. **Orchestration** - Coordinating agents through workflows

---

# 📚 FILE STUDY ORDER

Follow this order to understand the system from foundation to complete workflow:

```
PHASE 1: Foundation (Understand the building blocks)
    1. models/biogpt_wrapper.py    → Your ML model integration
    2. tools/clinical_tools.py     → Tools agents can use

PHASE 2: Agents (Learn how agents work)
    3. agents/triage_agent.py      → First agent, simplest example
    4. agents/diagnosis_agent.py   → Sequential pattern
    5. agents/treatment_agent.py   → Tool-heavy agent
    6. agents/scheduling_agent.py  → Parallel operations
    7. agents/documentation_agent.py → Data aggregation
    8. agents/followup_agent.py    → Loop pattern

PHASE 3: Coordination (See how it all connects)
    9. orchestrator.py             → Master coordinator
    10. demo.py                    → Running the system
```

---

# PHASE 1: FOUNDATION

---

## 📄 File 1: `models/biogpt_wrapper.py`

### Why This File is Important

This file **bridges your fine-tuned BioGPT model with the agent system**. It's crucial because:

1. **Encapsulates Model Complexity** - Agents don't need to know how to load/run the model
2. **Provides Consistent Interface** - All agents call the same `generate()` method
3. **Handles Device Management** - Automatically detects GPU/CPU
4. **Enables Testing** - `MockBioGPT` lets you test without loading the real model

### Key Concepts in This File

```python
# 1. MODEL ABSTRACTION
class BioGPTWrapper:
    def generate(self, prompt: str) -> str:
        # Agents just call this - they don't care about tokenization, etc.
        
# 2. ASYNC COMPATIBILITY (for Google ADK)
    async def agenerate(self, prompt: str) -> str:
        # ADK uses async/await, so we wrap sync calls
        
# 3. FACTORY PATTERN
def create_biogpt_wrapper(use_mock=False):
    # Easy switching between real and mock model
```

### When You'll Use This

- When agents need **medical reasoning** beyond what Gemini provides
- When generating **medical text** in the format your model was trained on
- During **testing** (use MockBioGPT to avoid loading heavy model)

In [ ]:
# Example: How BioGPT wrapper is used

from models.biogpt_wrapper import create_biogpt_wrapper

# For testing (no GPU needed)
biogpt = create_biogpt_wrapper(use_mock=True)

# For production (loads your fine-tuned model)
# biogpt = create_biogpt_wrapper(model_path="/path/to/your/model")

# Generate medical text
response = biogpt.generate("KEYWORDS: chest pain, shortness of breath\n\nTRANSCRIPTION:")
print(response)

---

## 📄 File 2: `tools/clinical_tools.py`

### Why This File is Important

**Tools extend what agents can DO**. Without tools, agents can only generate text. With tools, they can:

1. **Look up patient records** → `lookup_patient()`
2. **Check drug safety** → `check_drug_interactions()`
3. **Schedule appointments** → `schedule_appointment()`
4. **Send alerts** → `send_alert()`
5. **Create documentation** → `generate_clinical_note()`

### Key Concepts in This File

```python
# TOOLS ARE JUST FUNCTIONS!
# Google ADK can call these when the agent decides to use them

def lookup_patient(patient_id: str) -> Dict[str, Any]:
    """
    The docstring is CRITICAL - ADK reads this to know:
    - What the tool does
    - What parameters it needs
    - What it returns
    """
    # In production: query your database
    # For demo: return mock data
    return {"patient_id": patient_id, "name": "John", ...}

# TOOL REGISTRY - Which tools each agent gets
def get_triage_tools():
    return [assess_symptoms, lookup_patient, send_alert]

def get_treatment_tools():
    return [lookup_patient, check_drug_interactions, generate_clinical_note]
```

### Google ADK Tool Pattern

```python
from google.adk.agents import Agent

# Tools are passed when creating the agent
agent = Agent(
    name="triage_agent",
    model="gemini-2.0-flash",
    instruction="You are a triage nurse...",
    tools=[assess_symptoms, lookup_patient, send_alert]  # ← Tools here!
)
```

In [ ]:
# Example: Using tools directly

from tools.clinical_tools import (
    lookup_patient,
    check_drug_interactions,
    schedule_appointment
)

# Look up a patient
patient = lookup_patient("P001")
print("Patient:", patient)

# Check drug interactions
safety = check_drug_interactions(
    proposed_medications=["Aspirin"],
    current_medications=["Warfarin"],
    allergies=["Penicillin"]
)
print("Safety check:", safety)

# Schedule an appointment
appointment = schedule_appointment(
    patient_id="P001",
    appointment_type="follow_up",
    urgency="routine"
)
print("Appointment:", appointment)

---

# PHASE 2: AGENTS

Now that you understand the foundation, let's look at the agents.

---

## 📄 File 3: `agents/triage_agent.py`

### Why This File is Important

**This is the SIMPLEST agent** - study it first to understand the pattern:

1. **Entry Point** - First agent patients interact with
2. **Shows ADK Agent Structure** - instruction + tools pattern
3. **Demonstrates Decision Making** - urgency levels, red flags
4. **Prepares Data for Next Agent** - structured output

### Key Concepts in This File

```python
# 1. THE INSTRUCTION (System Prompt)
TRIAGE_INSTRUCTION = """
You are an experienced triage nurse AI...

## URGENCY LEVELS
- Level 1: IMMEDIATE...
- Level 2: EMERGENCY...

## RED FLAGS - ALWAYS ESCALATE
- Chest pain...
"""
# This defines the agent's PERSONALITY and BEHAVIOR

# 2. AGENT CREATION (Factory Pattern)
def create_triage_agent(model="gemini-2.0-flash"):
    return Agent(
        name="triage_agent",
        model=model,
        instruction=TRIAGE_INSTRUCTION,
        tools=get_triage_tools()
    )

# 3. HELPER FUNCTIONS
def check_red_flags(symptoms: str) -> Dict:
    # Quick safety check before full assessment
    
def format_triage_result(...) -> Dict:
    # Structured output for next agent
```

### The Flow

```
Patient Symptoms → Triage Agent → Urgency Level + Assessment → Diagnosis Agent
```

In [ ]:
# Example: Creating and understanding the Triage Agent

from agents.triage_agent import (
    create_triage_agent,
    check_red_flags,
    TRIAGE_INSTRUCTION
)

# See the instruction (first 500 chars)
print("TRIAGE INSTRUCTION (preview):")
print(TRIAGE_INSTRUCTION[:500])
print("...")

# Check for red flags
symptoms = "severe chest pain and difficulty breathing"
red_flags = check_red_flags(symptoms)
print(f"\nRed flags in '{symptoms}':")
print(red_flags)

---

## 📄 File 4: `agents/diagnosis_agent.py`

### Why This File is Important

**Demonstrates SEQUENTIAL PATTERN** - receives data from Triage:

1. **Receives Triage Output** - Uses previous agent's work
2. **Multi-Step Reasoning** - Differential diagnosis process
3. **BioGPT Integration** - Uses your model for medical reasoning
4. **Structured Medical Output** - Differential diagnoses, investigations

### Key Concepts

```python
# SEQUENTIAL PATTERN: Input comes from previous agent
def build_clinical_picture(triage_result, patient_data):
    """
    Combines:
    - Triage assessment (from Triage Agent)
    - Patient history (from database via tools)
    - Current symptoms
    """
    return {
        "chief_complaint": triage_result["chief_complaint"],
        "urgency": triage_result["urgency_level"],
        "medical_history": patient_data["history"],
        ...
    }

# BIOGPT FOR MEDICAL REASONING
async def generate_differential_with_biogpt(clinical_picture, biogpt):
    prompt = f"KEYWORDS: differential diagnosis...\nTRANSCRIPTION:"
    return await biogpt.agenerate(prompt)
```

### The Flow

```
Triage Result → Diagnosis Agent → Differential Diagnoses + Investigations → Treatment Agent
```

---

## 📄 File 5: `agents/treatment_agent.py`

### Why This File is Important

**Demonstrates TOOL-HEAVY AGENT** - safety is critical:

1. **Safety First** - MUST check allergies and interactions
2. **Heavy Tool Usage** - Multiple tool calls per request
3. **Validation Pattern** - Pre and post-generation checks
4. **Multi-Format Output** - Clinical plan + Patient education

### Key Concepts

```python
# SAFETY GATE - Critical checkpoint before treatment
def run_safety_checks(proposed_medications, patient_data):
    """
    1. Check allergies
    2. Check drug interactions
    3. Check special populations (pregnancy, elderly)
    
    Returns: {passed: True/False, warnings: [...], blockers: [...]}
    """
    
# If safety fails, treatment is BLOCKED
if not safety_result["passed"]:
    return {"status": "safety_hold", "message": "Physician review required"}
```

### The Flow

```
Diagnosis Result → Safety Checks → Treatment Plan → [Documentation + Scheduling]
                        ↓
                   If fails → STOP (Physician Review)
```

---

## 📄 File 6: `agents/scheduling_agent.py`

### Why This File is Important

**Demonstrates PARALLEL OPERATIONS**:

1. **Multiple Resources** - Check multiple provider schedules
2. **Retry Logic** - If slot unavailable, try next
3. **External Integration** - Would connect to real calendar APIs
4. **Runs in PARALLEL** with Documentation Agent

### Key Concepts

```python
# PARALLEL: This agent runs AT THE SAME TIME as Documentation
# In orchestrator.py:
doc_task = self._run_documentation(data)
sched_task = self._run_scheduling(data)
doc_result, sched_result = await asyncio.gather(doc_task, sched_task)  # PARALLEL!

# RETRY PATTERN (Loop within agent)
async def schedule_all_appointments(requirements, patient_id):
    for req in requirements:
        attempts = 0
        while attempts < MAX_ATTEMPTS:
            result = schedule_appointment(...)
            if result["status"] == "success":
                break
            attempts += 1  # Try next slot
```

---

## 📄 File 7: `agents/documentation_agent.py`

### Why This File is Important

**Demonstrates DATA AGGREGATION**:

1. **Pulls from ALL agents** - Triage, Diagnosis, Treatment, Scheduling
2. **Multiple Output Formats** - SOAP, Progress Notes, Discharge Summary
3. **Template-Based Generation** - Ensures consistent structure
4. **Compliance** - Medical documentation standards

### Key Concepts

```python
# AGGREGATION - Combine data from all sources
def aggregate_clinical_data(triage_result, diagnosis_result, treatment_result):
    return {
        "subjective": {...from triage...},
        "objective": {...from diagnosis...},
        "assessment": {...from diagnosis...},
        "plan": {...from treatment...}
    }

# TEMPLATE-BASED OUTPUT
def generate_soap_note(aggregated_data):
    """
    S - Subjective (what patient reports)
    O - Objective (what we observe)
    A - Assessment (our diagnosis)
    P - Plan (what we'll do)
    """
```

---

## 📄 File 8: `agents/followup_agent.py`

### Why This File is Important

**Demonstrates LOOP PATTERN** - continuous monitoring:

1. **Stateful** - Maintains monitoring state across cycles
2. **Periodic Execution** - Runs on schedule (daily, weekly)
3. **Alert Generation** - Triggers when thresholds exceeded
4. **Re-engagement** - Contacts patients who fall off track

### Key Concepts

```python
# LOOP STATE - Track what we're monitoring
class MonitoringState:
    active_monitors: Dict[str, Dict]  # patient_id -> monitoring config
    
    def add_patient(self, patient_id, treatment_plan):
        # Start monitoring this patient
        
    def get_due_for_check(self) -> List[str]:
        # Which patients need checking today?

# THE LOOP - One iteration
def run_monitoring_check(patient_id):
    """
    1. Assess current status
    2. Check alert conditions
    3. Generate outreach if needed
    4. Schedule next check
    5. REPEAT (when due again)
    """

# ALERTS - Trigger other agents
if alert_severity == "urgent":
    send_alert(patient_id, message, "urgent")
    # This might trigger Triage Agent to re-assess!
```

### The Loop Pattern Visualized

```
              ┌─────────────────────────────────┐
              │                                 │
              ▼                                 │
    [Check Patient] → [Assess Status] → [Generate Alerts?] → [Schedule Next Check]
              │                │                              │
              │                ▼                              │
              │         If urgent → Alert Triage Agent        │
              │                                               │
              └───────────── Wait until next_check_date ──────┘
```

---

# PHASE 3: COORDINATION

Now let's see how everything connects together.

---

## 📄 File 9: `orchestrator.py`

### Why This File is Important

**THIS IS THE BRAIN** - coordinates everything:

1. **Creates All Agents** - Initializes the 6 department agents
2. **Manages Workflows** - Sequential, Parallel, Loop execution
3. **Uses ADK Runner** - `InMemoryRunner` for agent execution
4. **Integrates BioGPT** - Hybrid model orchestration
5. **Provides Observability** - Status, history, metrics

### Key Concepts

```python
class ClinicalCareOrchestrator:
    def __init__(self):
        # 1. Initialize BioGPT
        self.biogpt = create_biogpt_wrapper(...)
        
        # 2. Create all agents
        self.agents = {
            "triage": create_triage_agent(),
            "diagnosis": create_diagnosis_agent(),
            ...
        }
        
        # 3. Create runners (ADK pattern)
        self.runners = {
            name: InMemoryRunner(agent=agent)
            for name, agent in self.agents.items()
        }
    
    async def run_full_workflow(self, patient_data):
        # SEQUENTIAL
        triage_result = await self._run_triage(patient_data)
        diagnosis_result = await self._run_diagnosis({**patient_data, **triage_result})
        treatment_result = await self._run_treatment({...})
        
        # PARALLEL
        doc_result, sched_result = await asyncio.gather(
            self._run_documentation(...),
            self._run_scheduling(...)
        )
        
        # LOOP INITIALIZATION
        followup_result = await self._setup_followup(...)
```

### The Complete Workflow

```
Patient Data
     │
     ▼
┌─────────┐    ┌───────────┐    ┌───────────┐
│ TRIAGE  │───▶│ DIAGNOSIS │───▶│ TREATMENT │
└─────────┘    └───────────┘    └───────────┘
                                      │
                    ┌─────────────────┼─────────────────┐
                    │                 │                 │
                    ▼                 ▼                 ▼
            ┌──────────────┐  ┌────────────┐    ┌──────────┐
            │DOCUMENTATION │  │ SCHEDULING │    │ FOLLOWUP │
            └──────────────┘  └────────────┘    └──────────┘
                 (PARALLEL)                       (LOOP)
```

In [ ]:
# Example: Understanding the orchestrator

# The three patterns in code:

# 1. SEQUENTIAL PATTERN
async def sequential_example():
    result1 = await agent1.process(data)
    result2 = await agent2.process({**data, **result1})  # Pass result forward
    result3 = await agent3.process({**data, **result1, **result2})
    return result3

# 2. PARALLEL PATTERN  
async def parallel_example():
    import asyncio
    # Both run at the same time!
    result_a, result_b = await asyncio.gather(
        agent_a.process(data),
        agent_b.process(data)
    )
    return result_a, result_b

# 3. LOOP PATTERN
async def loop_example():
    while monitoring_active:
        for patient in get_due_patients():
            await check_patient(patient)
        await sleep_until_next_cycle()

---

## 📄 File 10: `demo.py`

### Why This File is Important

**Shows how to RUN the system**:

1. **Entry Point** - How to start the application
2. **Example Patient Data** - What input looks like
3. **Output Display** - What results look like
4. **Command Line Interface** - Different modes (full, triage-only, etc.)

### Usage

```bash
# Run full demo with mock BioGPT
python demo.py

# Run with your real BioGPT model
python demo.py --biogpt-path /path/to/your/model

# Quick triage only
python demo.py --triage-only

# Show pattern explanations
python demo.py --show-patterns
```

---

# 🎓 SUMMARY: What Each File Teaches

| File | Key Learning | Pattern Demonstrated |
|------|--------------|---------------------|
| `biogpt_wrapper.py` | Model abstraction, async compatibility | Wrapper Pattern |
| `clinical_tools.py` | Tool definitions, ADK tool format | Tool Pattern |
| `triage_agent.py` | ADK Agent structure, simple agent | Agent Creation |
| `diagnosis_agent.py` | Using previous agent's output | **SEQUENTIAL** |
| `treatment_agent.py` | Safety checks, heavy tool usage | Validation Pattern |
| `scheduling_agent.py` | Multiple operations together | **PARALLEL** |
| `documentation_agent.py` | Combining all data sources | Aggregation Pattern |
| `followup_agent.py` | Continuous monitoring | **LOOP** |
| `orchestrator.py` | Coordinating all agents | All Patterns Combined |
| `demo.py` | Running the system | Application Entry |

---

# 🔧 GOOGLE ADK KEY CONCEPTS

### 1. Agent Creation
```python
from google.adk.agents import Agent

agent = Agent(
    name="my_agent",           # Unique identifier
    model="gemini-2.0-flash",  # LLM to use
    instruction="...",         # System prompt (personality)
    tools=[func1, func2]       # Functions agent can call
)
```

### 2. Running Agents
```python
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=agent)
response = await runner.run(
    session_id="unique_session",
    prompt="User's message"
)
```

### 3. Tools (Functions)
```python
def my_tool(param1: str, param2: int = 10) -> Dict[str, Any]:
    """
    Tool description - ADK reads this!
    
    Args:
        param1: Description of param1
        param2: Description of param2
        
    Returns:
        Description of return value
    """
    return {"result": "..."}
```

### 4. Sessions (Memory)
```python
# Same session_id = conversation continues
await runner.run(session_id="patient_001", prompt="First message")
await runner.run(session_id="patient_001", prompt="Follow up")  # Remembers context!

# Different session_id = new conversation
await runner.run(session_id="patient_002", prompt="Different patient")
```

---

# ✅ NEXT STEPS

1. **Study files in order** (Phase 1 → Phase 2 → Phase 3)
2. **Run the demo** to see it in action
3. **Integrate your BioGPT model** by updating the path
4. **Add more tools** for your specific use cases
5. **Implement evaluation metrics** for your capstone

Good luck with your project, Wagdy! 🎓